## Veiew DB in panadas

In [2]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
print(cols)
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
res = results[(results['ticker']!='None')]
res[['id', 'strategy_type', 'tab_name', 'er', 'ticker','trigger_price', 'strike_price', 'strike_buy','strike_sell','estimated_premium', 'last_price_when_checked',
         'timestamp_of_price_when_last_checked','scrape_date', 'timestamp_of_trigger','strategy_status', 'price_when_triggered', 'price_when_order_placed','premium_when_last_checked']].tail(14)

['id', 'scrape_date', 'strategy_type', 'tab_name', 'ticker', 'er', 'INTEGER', 'trigger_price', 'strike_price', 'strike_buy', 'strike_sell', 'estimated_premium', 'last_price_when_checked', 'timestamp_of_price_when_last_checked', 'item_id', 'options_expiry_date', 'date_info', 'timestamp_of_trigger', 'strategy_status', 'price_when_triggered', 'price_when_order_placed', 'premium_at_order', 'premium_when_last_checked', 'timestamp_of_order']


,id,strategy_type,tab_name,er,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,scrape_date,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked
95,297,Bear Call,Mild Risk 95-97% accuracy > longer expiry,0.0,IWM,198.63,sell 207.0 - buy 217.0,217.0,207.0,168.0,194.10,2025-04-24T23:44:06.101103,2025-04-24T05:15:15.928614,None,None,NaN,None,NaN
96,298,Bear Call,Mild Risk 95-97% accuracy > longer expiry,0.0,NKE,59.65,sell 62.0 - buy 72.0,72.0,62.0,190.0,58.09,2025-04-24T23:44:06.115211,2025-04-24T05:15:16.003685,None,None,NaN,None,NaN
97,299,Bear Call,Mild Risk 95-97% accuracy > longer expiry,0.0,NVDA,106.08,sell 115.0 - buy 125.0,125.0,115.0,172.0,107.68,2025-04-24T23:44:06.122516,2025-04-24T05:15:16.092935,2025-04-24T16:37:52.194131,premium too low,106.16,None,0.0
99,301,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,0.0,JNJ,152.99,sell 147.0 - buy 135.0,135.0,147.0,82.0,154.93,2025-04-24T23:44:06.192253,2025-04-24T05:15:29.804157,None,None,NaN,None,NaN
100,302,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,0.0,UNH,419.92,sell 390.0 - buy 380.0,380.0,390.0,155.0,424.50,2025-04-24T23:44:06.203971,2025-04-24T05:15:29.935430,None,None,NaN,None,NaN
102,304,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,JNJ,152.13,sell 145.0 - buy 135.0,135.0,145.0,111.0,154.93,2025-04-24T23:44:06.184405,2025-04-24T05:15:37.698425,None,None,NaN,None,NaN
103,305,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,0.0,UNH,420.4,sell 390.0 - buy 380.0,380.0,390.0,135.0,424.50,2025-04-24T23:44:06.212167,2025-04-24T05:15:41.357254,None,None,NaN,None,NaN
104,306,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0.0,GOOGL,166.34,sell 175.0 - buy 185.0,185.0,175.0,167.0,161.86,2025-04-25T23:46:01.608180,2025-04-25T07:25:42.076518,None,None,NaN,None,NaN
105,307,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0.0,META,553.6,sell 600.0 - buy 610.0,610.0,600.0,282.0,547.56,2025-04-25T23:46:01.613140,2025-04-25T07:25:42.164857,None,None,NaN,None,NaN
106,308,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0.0,MSFT,398.38,sell 420.0 - buy 430.0,430.0,420.0,175.0,391.90,2025-04-25T23:46:01.617239,2025-04-25T07:25:42.272876,None,None,NaN,None,NaN


In [15]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
print(cols)
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
res = results[(results['ticker']!='None') & (results['strategy_status']=='order placed')]
res[['id', 'strategy_type', 'tab_name', 'er', 'ticker','trigger_price', 'strike_price', 'strike_buy','strike_sell','estimated_premium', 'options_expiry_date',
         'timestamp_of_price_when_last_checked','scrape_date', 'timestamp_of_trigger','strategy_status', 'price_when_triggered', 'price_when_order_placed','premium_when_last_checked']].tail(9)

['id', 'scrape_date', 'strategy_type', 'tab_name', 'ticker', 'er', 'INTEGER', 'trigger_price', 'strike_price', 'strike_buy', 'strike_sell', 'estimated_premium', 'last_price_when_checked', 'timestamp_of_price_when_last_checked', 'item_id', 'options_expiry_date', 'date_info', 'timestamp_of_trigger', 'strategy_status', 'price_when_triggered', 'price_when_order_placed', 'premium_at_order', 'premium_when_last_checked', 'timestamp_of_order']


,id,strategy_type,tab_name,er,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,options_expiry_date,timestamp_of_price_when_last_checked,scrape_date,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked
6,7,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,NKE,55,sell 49.0 - buy 40.0,40.0,49.0,100.0,2025-05-02,2025-04-11T20:30:59.868610,2025-04-11T15:50:09.447411,2025-04-11T16:26:34.591504,order placed,53.81,None,109.949105
8,180,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,NaN,GOOGL,159,sell 175.0 - buy 185.0,185.0,175.0,10.0,2025-04-25,2025-04-14T20:31:07.118067,2025-04-14T07:32:32.512393,2025-04-14T15:26:45.698568,order placed,160.35,None,17.500000
9,181,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,NaN,SPY,537.02,sell 560.0 - buy 570.0,570.0,560.0,190.0,2025-04-25,2025-04-14T20:31:07.122358,2025-04-14T07:32:32.610704,2025-04-14T13:43:01.832916,order placed,540.32,None,239.500000
27,199,Bull Put,Mild Risk 95-97% accuracy > longer expiry,NaN,META,518.65,sell 475.0 - buy 465.0,465.0,475.0,273.0,2025-05-09,2025-04-15T23:44:17.714278,2025-04-15T04:58:35.221500,2025-04-15T18:14:01.482272,order placed,518.40,None,1400.000000
38,240,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,META,508.07,sell 465.0 - buy 455.0,455.0,465.0,230.0,2025-05-09,2025-04-16T21:14:13.546153,2025-04-16T07:52:15.912922,2025-04-16T13:31:03.469463,order placed,507.36,None,242.500000
39,241,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,0.0,MSFT,375.37,sell 355.0 - buy 345.0,345.0,355.0,250.0,2025-05-09,2025-04-16T21:14:13.564535,2025-04-16T07:52:19.547455,2025-04-16T17:33:32.429387,order placed,375.21,None,262.500000
58,260,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0.0,NKE,57.29,sell 60.0 - buy 70.0,70.0,60.0,110.0,2025-05-09,2025-04-22T23:46:13.320511,2025-04-22T10:47:09.950276,2025-04-22T15:04:58.565379,order placed,57.31,None,111.500000


## Delete ALL rows in SQLite database

In [5]:
stop

NameError: name 'stop' is not defined

In [ ]:

import sqlite3

conn = sqlite3.connect('option_strategies.db')
c = conn.cursor()

# Delete all rows from table
c.execute('DELETE FROM option_strategies;',)

print('We have deleted', c.rowcount, 'records from the table.')

# Commit the changes to db			
conn.commit()
# Close the connection
conn.close()

We have deleted 20 records from the table.


## Delete table

In [ ]:

# importing sqlite module 
import sqlite3 
  
# create connection to the  
# database geek 
connection = sqlite3.connect('option_strategies.db') 
  
# drop table 
connection.execute("DROP TABLE option_strategies") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully


## Update Row

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (55, 7))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## update estimated premium

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET estimated_premium=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (10, 180))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## update trigger price

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (159, 180))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## update ticker symbol

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET ticker=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, ('JNJ', 192))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## Change columns entries 'strategy_status'

In [ ]:
import sqlite3

def update_strategy_status():
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Define the SQL update query
        update_query = """
        UPDATE option_strategies
        SET strategy_status = 'triggered'
        WHERE strategy_status = 'premium too low';
        """
        
        # Execute the update query
        cursor.execute(update_query)
        conn.commit()
        
        # Report the number of rows updated
        print(f"{cursor.rowcount} rows updated.")
        
    except sqlite3.Error as error:
        print("Error while updating SQLite table:", error)
    finally:
        # Ensure that the connection is closed
        if conn:
            conn.close()

if __name__ == "__main__":
    update_strategy_status()



1 rows updated.


 ## Change columns entries 'scrape_date'

In [ ]:
import sqlite3

def update_scrape_date():
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Update query: change scrape_date to '2025-04-06'
        # where the date portion of scrape_date is '2025-04-03'
        update_query = """
        UPDATE option_strategies
        SET scrape_date = '2025-04-11'
        WHERE DATE(scrape_date) = '2025-04-10';  
        """
        
        # Execute the update query
        cursor.execute(update_query)
        conn.commit()
        
        # Output the number of rows updated
        print(f"{cursor.rowcount} rows updated.")
        
    except sqlite3.Error as error:
        print("Error updating scrape_date:", error)
    finally:
        # Ensure the connection is closed
        if conn:
            conn.close()

if __name__ == "__main__":
    update_scrape_date()



13 rows updated.


## Add New column

In [ ]:
import sqlite3
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def add_premium_column():
    """Add a 'timestamp_of_order ' column to the option_strategies table."""
    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Check if the column already exists
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [col[1] for col in cursor.fetchall()]
        
        if 'timestamp_of_order ' in columns:
            logger.info("Column 'timestamp_of_order  already exists in the table.")
            return
        
        # Add the new column - using REAL type for storing decimal/float values
        cursor.execute("""
            ALTER TABLE option_strategies 
            ADD COLUMN timestamp_of_order DATETIME
        """)
        
        conn.commit()
        logger.info("Successfully added 'timestamp_of_order' column to option_strategies table.")
        
    except sqlite3.Error as e:
        logger.error(f"SQLite error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    add_premium_column()

2025-04-06 08:14:48,843 - ERROR - SQLite error: no such table: option_strategies


## Query a section of the database

In [ ]:


start_date = '2025-03-29'
dat = sqlite3.connect('option_strategies.db')
query_text = f"""
    SELECT * FROM option_strategies 
    WHERE date(scrape_date) = '{start_date}'
    AND strategy_status = 'triggered'
    """
query = dat.execute(query_text)
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

OperationalError: no such table: option_strategies

## Delete all row except 1

In [ ]:
import sqlite3

# Connect to your database
conn = sqlite3.connect('option_strategies.db')
cur= conn.cursor()

# Delete all rows except the one where id = 312
cur.execute("DELETE FROM option_strategies WHERE id != ?", (312,))

# Commit changes and close connection
conn.commit()
conn.close()

print("Rows deleted except the one with id = 312.")

Rows deleted except the one with id = 312.


## Delete rows from scrape date

In [ ]:
import sqlite3

date_to_delete = '2025-04-16'

# Connect to your database
conn = sqlite3.connect('option_strategies.db')
cur = conn.cursor()

# Delete rows for the specific date
cur.execute("DELETE FROM option_strategies WHERE DATE(scrape_date) = ?", (date_to_delete,))

# Get number of rows deleted
rows_deleted = cur.rowcount

# Commit changes and close connection
conn.commit()
conn.close()

print(f"{rows_deleted} rows deleted.")



20 rows deleted.


## Change Ticker symbol

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET ticker=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, ('WMT', 179))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET ticker=?, trigger_price=?, strike_buy=?, strike_sell=? WHERE id = ?'
try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, ('BBY', 650, 680, 690, 312))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)